# Python Code to Process Vaccine Distribution Data from CDC

Raw data is from CDC's vaccine data tracker and daily updated on: 
https://covid.cdc.gov/covid-data-tracker/#vaccinations

In [ ]:
import os
import re
import pandas as pd

In [ ]:
# load the previous data and save a backup
file_dir = '/Users/xywu/Documents/HPC_datahub/vaccine/'
orig_df = pd.read_csv(file_dir + 'vaccine_distribution.csv')
orig_df.to_csv(file_dir + 'vaccine_distribution_last_week.csv', index=False)

In [ ]:
# extract target filenames
data_dir = file_dir + 'temp_data'
file_all = os.listdir(data_dir)

# get rid of '.DS_Store' in the folder if any
try:
    file_all.remove('.DS_Store')
except ValueError:
    pass

# extract dates
dts = []
for i in range(len(file_all)):
    temp_file = file_all[i].split('-')
    temp_date = re.sub('\..*', '', temp_file[1])
    dts.append(int(temp_date))
dts

In [ ]:
# reorder the filename list
dts_order = [dts.index(x) for x in sorted(dts)]
file_lst = [file_all[dts_order[x]] for x in range(len(dts_order))]

In [ ]:
# read all data into a list of dataframes
df_lst = []
for i in range(len(file_lst)):
    df_lst.append(pd.read_csv(file_dir + 'temp_data/' + file_lst[i]))

In [ ]:
# rename columns
dts = sorted(dts)
for i in range(len(dts)):
    temp_colnames = [df_lst[i].columns[0]] + [str(dts[i])+'_'+df_lst[i].columns[x] \
                                              for x in range(1, df_lst[0].shape[1])]
    df_lst[i].columns = temp_colnames

In [ ]:
# join the seperate dataframes into one dataframe
df = df_lst[0]
for i in range(1, len(df_lst)):
    df = df.merge(right=df_lst[i], on='State/Territory/Federal Entity', how='outer')
# check
#print(df.shape)
#df.columns

In [ ]:
# remove the jurisdictions outside of the 50 continent states

# identify the rows outside of the 50 continent states
to_rm = ['American Samoa', 'Bureau of Prisons', 'Dept of Defense', \
         'Federated States of Micronesia', 'Guam', 'Indian Health Svc',\
         'Marshall Islands', 'Northern Mariana Islands', 'Puerto Rico',\
         'Republic of Palau', 'Veterans Health', 'Virgin Islands']

# remove
to_rm_id = [df.index[df['State/Territory/Federal Entity']==x].tolist()[0] for x in to_rm]
df = df.drop(to_rm_id,  axis=0)

In [ ]:
# rename df columns
col1 = [str.replace(x.lower(), ' ', '_') for x in list(df.columns)]
m = re.compile('(\d+)(.*)')
col2 = [m.search(x).group(1) + '_distribution' + m.search(x).group(2) \
            for x in col1[1:]]
df.columns = [col1[0]] + col2

In [ ]:
# merge with previous data
final = orig_df.merge(right=df, on='state/territory/federal_entity', how='outer')

In [ ]:
# output file
final.to_csv(file_dir + '/' + 'vaccine_distribution.csv', index=False)

In [ ]:
# manually inspect
final